In [1]:
import collections
import copy
import sys
import time
from random import seed

import matplotlib.pyplot as plt
import numpy as np
from matplotlib import animation
from torch import optim

import dataset
import evaluation
from GaussianDiffusion import GaussianDiffusionModel, get_beta_schedule
from helpers import *
from UNet import UNetModel, update_ema_params

torch.cuda.empty_cache()

ROOT_DIR = "./"


In [2]:
args = {
  "img_size": [
    256,
    256
  ],
  "Batch_Size": 16,
  "EPOCHS": 3000,
  "T": 1000,
  "base_channels": 128,
  "beta_schedule": "linear",
  "channel_mults": "",
  "loss-type": "l2",
  "loss_weight": "none",
  "train_start": True,
  "lr": 1e-4,
  "random_slice": False,
  "sample_distance": 800,
  "weight_decay": 0.0,
  "save_imgs": False,
  "save_vids": True,
  "dropout": 0,
  "attention_resolutions": "16,8",
  "num_heads": 2,
  "num_head_channels": -1,
  "noise_fn": "simplex",
  "dataset": "mri"
}

In [6]:
in_channels=3
unet = UNetModel(
        args['img_size'][0], args['base_channels'], channel_mults=args['channel_mults'], in_channels=in_channels
        )

In [8]:
x = torch.randn(10, 3, 256, 256)
t_batch = torch.tensor([1], device=x.device).repeat(x.shape[0])
y_hat = unet(x, t_batch)

In [9]:
betas = get_beta_schedule(args['T'], args['beta_schedule'])

diff = GaussianDiffusionModel(
        args['img_size'], betas, loss_weight=args['loss_weight'],
        loss_type=args['loss-type'], noise=args["noise_fn"], img_channels=in_channels
        )

In [17]:
# t_batch.shape

In [19]:
# diff.noise_fn(x, t_batch)